# cbvLang
cbvEval 노트북에서 정의했던 내용을 그대로 가져와서
Tm이라는 타입의 이름을 Expr로 바꾸고
거기에 몇 가지 프로그래밍 요소를 추가했다.

In [1]:
-- 변수 이름은 문자열로 나타낸다
type Nm = String

-- 장난감 함수형 언어 = 람다식 + ...
data Expr
  = Var Nm          -- x, y, z, ...
  | I Int           -- 0, -1, 1, -2, 2, -3, 3, ...
  | B Bool          -- False, True
  | Lam Nm Expr      -- \x.e
  | Rec Nm Expr      -- rec f e
  | App Expr Expr      -- e1 e2
  | If Expr Expr Expr  -- if e then e1 else e2
  | Let Nm Expr Expr  -- let x=e2 in e
  | Add Expr Expr     -- e1 + e2
  | Mul Expr Expr     -- e1 * e2
  | Neg Expr         -- - e
  | Eq Expr Expr      -- e1 == e2
  | Ne Expr Expr      -- e1 /= e2
  | Lt Expr Expr      -- e1 <  e2
  | Gt Expr Expr      -- e1 >  e2
  | Le Expr Expr      -- e1 <= e2
  | Ge Expr Expr      -- e1 >= e2
  | And Expr Expr     -- e1 && e2
  | Or Expr Expr      -- e1 || e2
  | Not Expr         -- not e
  deriving (Show, Eq)

In [2]:
import IHaskell.Display

dispExpr t = Display [html("<code>"++dpExpr t++"</code>")]

dpt t@(Lam{}) = paren (dpExpr t)
dpt t@(Let{}) = paren (dpExpr t)
dpt t@(Rec{}) = paren (dpExpr t)
dpt t         = dpExpr t

dps s@(I _)   = dpExpr s
dps s@(B _)   = dpExpr s
dps s@(Var{}) = dpExpr s
dps s         = paren (dpExpr s)

dpe = dps

dpExpr (Var x) = x
dpExpr (Lam x t) = "λ" ++ x ++ "." ++ dpExpr t
dpExpr (App t s) = dpt t ++ " " ++ dps s
dpExpr (I n) = show n
dpExpr (B b) = show b
dpExpr (Add e1 e2) = dpe e1++" + "++dpe e2
dpExpr (Mul e1 e2) = dpe e1++" * "++dpe e2
dpExpr (Neg e) = "-"++dpe e
dpExpr (Eq e1 e2) = dpe e1++" == "++dpe e2
dpExpr (Ne e1 e2) = dpe e1++" /= "++dpe e2
dpExpr (Lt e1 e2) = dpe e1++" < "++dpe e2
dpExpr (Gt e1 e2) = dpe e1++" > "++dpe e2
dpExpr (Le e1 e2) = dpe e1++" <= "++dpe e2
dpExpr (Ge e1 e2) = dpe e1++" >= "++dpe e2
dpExpr (And e1 e2) = dpe e1++" && "++dpe e2
dpExpr (Or e1 e2) = dpe e1++" || "++dpe e2
dpExpr (Not e) = "not "++ dpe e
dpExpr (Let x e2 e) = "let "++x++" = "++dpt e2++" in "++dpExpr e
dpExpr (Rec f e) = "rec "++f++" "++dpExpr e
dpExpr (If e e1 e2) = "if "++dpe e++" then "++dpe e1++" else "++dpe e2

paren s = "(" ++ s ++ ")"

In [3]:
idExpr = Lam "x" (Var "x")

putStrLn $ dpExpr $ Lam "y" (App idExpr (App idExpr (Var "y")))

dispExpr $ Lam "y" (App idExpr (App idExpr (Var "y")))

λy.(λx.x) ((λx.x) y)

λy.(λx.x) ((λx.x) y)

In [4]:
putStrLn $ dpExpr $ I 3 `Add` I 4
putStrLn $ dpExpr $ Let "id" idExpr (App (Var "id") (Var "id"))

3 + 4

let id = (λx.x) in id id

# 장난감 함수형 언어의 call-by-value evaluation
call-by-value evaluation (줄여서 CVB evaluation)은
적극적 계산법(eager evaluation)이라고 부르기도 한다.

람다식에 대한 적극적 계산법의 big-step 실행의미를 추론규칙 형태로 정의했던 규칙에 덧붙여
새로 장난감 함수형 언어에 추가된 요소들에 대한 규칙들 중요한 것들을 추려서 제시하였다.

$\displaystyle\begin{array}{rcrcl}
\sigma&\in&\textit{Env} &=&
\textit{Nm} \longrightarrow^{\hspace{-2.7ex}\textrm{fin}} \textit{Val} \\
v&\in&\textit{Val} &=&  \textit{Int} ~+~ \textit{Bool} ~+~ \textit{Expr}_V \times Env \\
n&\in&\textit{Int}&=& \{\ldots,-2,-1,0,1,2,\ldots\} \\
b&\in&\textit{Bool}&=& \{\textbf{False},\textbf{True}\} \\
\end{array}$

$\displaystyle\begin{array}{rcl}
\textit{Expr}_V &\subset& Expr\\
\textit{Expr}_V &=& \{ \lambda x.e \mid x\in\textit{Nm}, e\in\textit{Expr}\}
              \cup \{ \textbf{rec}~f~\lambda x.e \mid f\in\textit{Nm}, x\in\textit{Nm}, e\in\textit{Expr}\}
\end{array}$

$~$
$~$

$\displaystyle\Downarrow ~:~ \textit{Expr}\times \textit{Env} \to \textit{Val} $

$~$

$\displaystyle\frac{}{(x, \sigma) \Downarrow \sigma(x)} 
\quad
 \displaystyle\frac{}{(\lambda x.e, \sigma) \Downarrow \langle\lambda x.e, \sigma\rangle}
\quad
 \displaystyle\frac{}{(\textbf{rec}~f~\lambda x.e, \sigma) \Downarrow \langle\textbf{rec}~f~\lambda x.e, \sigma\rangle}
$

$~$

$
\displaystyle\frac{}{(n, \sigma) \Downarrow n}
\qquad
\displaystyle\frac{}{(\textbf{True}, \sigma) \Downarrow \textbf{True}}
\quad
\displaystyle\frac{}{(\textbf{False}, \sigma) \Downarrow \textbf{False}}
$

$~$

$\displaystyle\frac{
   ((\lambda x.e)~e_2, \sigma) \Downarrow v}{
   (\textbf{let}~x=e_2~e, \sigma) \Downarrow v}
\qquad
 \displaystyle\frac{~
   \begin{array}{l}
   (e_1,\sigma) \Downarrow \langle\lambda x.e, \sigma_1\rangle \\
   (e_2,\sigma) \Downarrow v_2 \\
   (e, \{x\mapsto v_2\}\sigma_1) \Downarrow v
   \end{array}~}{
   (e_1~e_2, \sigma) \Downarrow v}
\qquad
 \displaystyle\frac{~
   \begin{array}{l}
   (e_1,\sigma) \Downarrow \langle\textbf{rec}~f~\lambda x.e, \sigma_1\rangle \\
   (e_2,\sigma) \Downarrow v_2 \\
   (e, \{x\mapsto v_2,\;f\mapsto\langle\textbf{rec}~f~\lambda x.e,\sigma_1\rangle\}\sigma_1) \Downarrow v
   \end{array}~}{
   (e_1~e_2, \sigma) \Downarrow v}
$

$~$


$\displaystyle\frac{~
   (e,\sigma) \Downarrow \textbf{True} \quad
   (e_1,\sigma) \Downarrow v}{
   (\textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_2, \sigma) \Downarrow v}
\qquad
 \displaystyle\frac{~
   (e,\sigma) \Downarrow \textbf{False} \quad
   (e_2,\sigma) \Downarrow v}{
   (\textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_2, \sigma) \Downarrow v}
$

$~$

$\displaystyle\frac{~
   \begin{array}{l}
   (e_1,\sigma) \Downarrow n_1\\
   (e_2,\sigma) \Downarrow n_2\\
   n = n_1 \stackrel{\textit{Int}}{+} n_2
   \end{array}~}{
   (e_1 + e_2, \sigma) \Downarrow n}
\qquad\cdots\quad\cdots
\qquad
 \displaystyle\frac{~
   \begin{array}{l}
   (e_1,\sigma) \Downarrow b_1\\
   (e_2,\sigma) \Downarrow b_2\\
   b = b_1 \land b_2
   \end{array}~}{
   (b_1 \mathop{\mathsf{\&\!\!\&}\,} b_2, \sigma) \Downarrow b}
\qquad\cdots\quad\cdots
$

$~$

$
 \displaystyle\frac{~
   \begin{array}{l}
   (e_1,\sigma) \Downarrow v_1\\
   (e_2,\sigma) \Downarrow v_2\\
   v_1 \stackrel{\tau}{=} v_2
   \end{array}~}{
   (e_1 = e_2, \sigma) \Downarrow \textbf{True}}~~(\tau\in\{\textit{Int},\textit{Bool}\})
\quad
 \displaystyle\frac{~
   \begin{array}{l}
   (e_1,\sigma) \Downarrow v_1\\
   (e_2,\sigma) \Downarrow v_2\\
   v_1 \stackrel{\tau}{\neq} v_2
   \end{array}~}{
   (e_1 = e_2, \sigma) \Downarrow \textbf{False}}~~(\tau\in\{\textit{Int},\textit{Bool}\})
\qquad\cdots\quad\cdots
$

위의 표기법은 비유하자면 $v = f(x)$라고 쓰는 대신에 $f(x)$를 기계적인 규칙을 따라 계속 계산을 돌리다 보면 $v$라는 값에 도달한다는 의미로 $f(x) \Downarrow v$라는 식으로 표시하는 것이다. 그리고 바로 람다식의 적극적 계산법을 위한 기계적인 규칙이 위에 나와 있는 세 가지 규칙이다.
산술연산은 Int 타입의 정수값에 대해서만, 논리연산은 Bool 타입의 진리값에 대해서만, 비교연산은 Int 타입의 정수값끼리 또는 Bool 타입의 진리값끼리만 정의되어 있다. 여기서는 논리연산의 short circuit에 대해서는 일단 생각하지 않기로 하자.

In [5]:
-- finite mapping을 순서쌍 리스트 타입으로 정의
type Env = [(Nm, Val)]
-- Expr_V를 따로 정의하지 않고 그냥 Expr을 이용해 정의
data Val = VI Int | VB Bool | Cl Expr Env  deriving Show

-- simga(x)에 해당하는 것이 lookup x simga
lookup' x env  = v  where Just v = lookup x env

# hw3: 다음 인터프리터를 완성하라 (제출기한: 중간고사 전날 10/22일 밤)
인터프리터를 완성하고 팩토리알 프로그램을 Expr 문법으로 작성하여 eval로 실행시켜 보라.

In [6]:
eval :: Expr -> Env -> Val
eval (I n) _ = VI n
eval (B b) _ = VB b
eval v@(Lam x e)         env = Cl v env
eval v@(Rec f (Lam x e)) env = Cl v env
eval (Var x)     env = lookup' x env
eval (App e1 e2) env =
  let v2 = eval e2 env in
  case eval e1 env of
    Cl v1@(Lam x e)         env1 -> eval e ((x,v2):env1)
    Cl v1@(Rec f (Lam x e)) env1 -> undefined
eval (If e e1 e2) env = undefined
eval (Let x e2 e) env = undefined -- let x=e2 in e 는 (\x.e) e2 의 문법설탕(syntactic sugar)
eval (Add e1 e2)  env = VI (x+y)
  where
  VI x = eval e1 env
  VI y = eval e2 env
eval (And e1 e2)  env = undefined
eval (Or e1 e2)   env = undefined
eval (Not e)      env = undefined
eval (Mul e1 e2)  env = undefined
eval (Neg e)      env = undefined
eval (Eq e1 e2)   env = undefined
eval (Ne e1 e2)   env = undefined
eval (Lt e1 e2)   env = undefined
eval (Gt e1 e2)   env = undefined
eval (Le e1 e2)   env = undefined
eval (Ge e1 e2)   env = undefined

In [7]:
-- eval을 완성한 후 multExpr와 factExpr를 완성하여 7팩토리알을 실행해 보라
factExpr = Rec undefined undefined
eval (Let "fact" factExpr $ Var "fact" `App` I 7) []

Prelude.undefined
CallStack (from HasCallStack):
  error, called at libraries/base/GHC/Err.hs:79:14 in base:GHC.Err
  undefined, called at <interactive>:13:25 in interactive:Ghci115

In [8]:
:type eval
:type uncurry eval

eval (App (Lam "x" (Var "x")) (Lam "x" (Var "x"))) []

sigma = [("twice",Cl (Lam "f"(Lam "x"(App f(App f x)))) [])]
      where
       f = Var "f"
       x = Var "x"

twice = Var "twice"

eval (App twice twice) sigma

eval :: Expr -> Env -> Val

uncurry eval :: (Expr, Env) -> Val

Cl (Lam "x" (Var "x")) []

Cl (Lam "x" (App (Var "f") (App (Var "f") (Var "x")))) [("f",Cl (Lam "f" (Lam "x" (App (Var "f") (App (Var "f") (Var "x"))))) [])]

In [9]:
import Data.List (intersperse)
dpEnv env = "{"++ concat (intersperse ", " [x++" ↦ "++dpVal v | (x,v)<-env]) ++ "}"

dpVal (VI n) = show n
dpVal (VB b) = show b
dpVal (Cl t env) = "⟨"++dpExpr t++", "++dpEnv env++"⟩"

dispEnv env = Display[html $ "<code>"++dpEnv env++"</code>"]
dispVal v = Display[html $ "<code>"++dpVal v++"</code>"]

In [10]:
dispEnv sigma

{twice ↦ ⟨λf.λx.f (f x), {}⟩}

In [11]:
putStrLn . dpVal $ eval (App twice twice) sigma
dispVal $ eval (App twice twice) sigma

⟨λx.f (f x), {f ↦ ⟨λf.λx.f (f x), {}⟩}⟩

⟨λx.f (f x), {f ↦ ⟨λf.λx.f (f x), {}⟩}⟩

In [12]:
idExpr

Lam "x" (Var "x")

In [13]:
putStrLn . dpVal $ eval (I 3) sigma
dispVal $ eval (I 3) sigma

doubleExpr = Lam "x" (Var "x" `Add` Var "x")

dispVal $ eval (App idExpr (I 3)) sigma
dispVal $ eval (App doubleExpr (I 3)) sigma

3

3

3

6

In [14]:
putStrLn . dpVal $ eval (B True) sigma
dispVal $ eval (B True) sigma

True

True